In [1]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB # modele d'exemple a modifier
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle

In [3]:
# Import data
data = pd.read_csv('data/train_diabetes_health_indicators.csv')
data['Diabetes_012'] = data['Diabetes_012'].astype(int)
data = data.drop(columns=['Unnamed: 0'])
X = data.drop(columns=['Diabetes_012'])
y = data['Diabetes_012']


In [ ]:
Data Exploring

# Analyse data

data.head()

In [ ]:
data.shape

In [ ]:
data.duplicated().sum()

In [ ]:
data.describe()

In [ ]:
data.nunique()

In [ ]:
data.dtypes

In [ ]:
data["Diabetes_012"].value_counts().plot.pie( autopct='%1.1f%%',shadow=True, startangle=90);


In [ ]:
plt.figure(figsize = (20,10))
sns.heatmap(X.corr(),annot=True , cmap ='YlOrRd' )
plt.title("correlation of feature")
plt.show()

In [ ]:
# BOX plot
import matplotlib.pyplot as plt

for column in data:
        if(data[column].nunique()>3):
                plt.figure(figsize=(17,1))
                sns.boxplot(data=data, x=column)

In [ ]:
# Remove outliers 
data=data[data["BMI"]>45]
data=data[data["GenHlth"]>4]
data=data[data["MentHlth"]>10]
data=data[data["PhysHlth"]>5]


In [ ]:
### Feature Selection (Pearson Correlation) and split data

pearson_list = []
for l in list(X.columns):
    p,_ = pearsonr(X[l], y)
    pearson_list.append(abs(p))
    
p_value=pd.Series(pearson_list,index=X.columns)
p_value.sort_values(ascending=False,inplace=True)
high_score_features_pearson = list(p_value.index[:15]) #we took only 15 features
print(high_score_features_pearson)

X=X[high_score_features_pearson]

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# UNDER Sampling   equilibrage data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

random_under_sampler = RandomUnderSampler(random_state=42)
X_train, y_train =random_under_sampler.fit_resample(X_train, y_train)

In [ ]:
#show
y_train.value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Class Distribution after balancing')
plt.xlabel('Class Labels')
plt.ylabel('Count')

In [ ]:
### Normalisation
from sklearn.preprocessing import StandardScaler

# Supposons que X_train et X_test sont vos ensembles de données d'entraînement et de test respectivement

# Initialisez le StandardScaler
scaler = StandardScaler()

# Calculez la moyenne et l'écart type sur X_train
scaler.fit(X_train)

# Transformez X_train et X_test en utilisant la standardisation
X_train_standardized = scaler.transform(X_train)
X_test_standardized = scaler.transform(X_test)


In [ ]:
#fit
# Initialize the model
#---- à modifier en utilisant l'un des trois modeles : SVM, RandomForest et GradientBoosting ----
clf = SVC(kernel='rbf')

# Train the model
clf.fit(X_train, y_train)

In [ ]:
# Predict on test data
pred = clf.predict(X_test)

In [ ]:
# grid search 
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report





# Définir les paramètres à tester dans la recherche sur grille
param_grid = {
    #'C': [0.1, 1, 10, 100],
    #'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

# Créer un modèle SVM
svm_model = SVC()

# Créer un objet GridSearchCV
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, scoring='accuracy')

# Exécuter la recherche sur grille sur les données d'entraînement
grid_search.fit(X_train, y_train)

# Extraire les meilleurs paramètres
best_params = grid_search.best_params_
print("Meilleurs paramètres:", best_params)

# Extraire le meilleur modèle
best_model = grid_search.best_estimator_

# Faire des prédictions sur l'ensemble de test avec le meilleur modèle
y_pred = best_model.predict(X_test)

# Évaluer la précision du modèle
accuracy = best_model.score(X_test, y_test)
print("Accuracy:", accuracy)

# Autres métriques d'évaluation
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
# Créer un modèle SVM

svm_model = SVC() 

# Effectuer une validation croisée sur l'ensemble d'entraînement
cv_scores = cross_val_score(svm_model, X_train, y_train, cv=5, scoring='accuracy')

# Afficher les scores de validation croisée
print("Scores de validation croisée:", cv_scores)
print("Moyenne des scores de validation croisée:", cv_scores.mean())

In [ ]:
# Evaluate model predictions
print(f'Classification_report:\n {classification_report(y_test,pred)}')